<a href="https://colab.research.google.com/github/SunnieCoder/Machine-Learning-projects/blob/main/face_recognition_project%20-%20using%20Dlib%20and%20KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import math
!pip install pyheif
import pyheif

!pip install dlib
import dlib
!pip install face_recognition
!pip3 install scikit-learn
from sklearn import neighbors
import pickle
from PIL import Image, ImageDraw, ImageFont
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
import os

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'PNG', 'JPG', 'JPEG', 'HEIC', 'heic'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Train the model: a k-nearest neighbors classifier for face recognition
# :param path: directory that contains a sub-directory (named as the person name) for each known person.

def train(path, n_neighbors=None, knn_algo='ball_tree', verbose=True):

  encodeListKnown = [] # each image's encoding
  classNames = [] # person's name corresponding to the image

  # Loop through each person/className in the training set
  for class_dir in os.listdir(path):
    if not os.path.isdir(os.path.join(path, class_dir)):
      continue

    # Loop through each training image for the current person
    for img_path in image_files_in_folder(os.path.join(path, class_dir)):
      # skip the non-image file
      if img_path.split(".")[-1] not in ALLOWED_EXTENSIONS:
        continue

      # handle the ".HEIC" training image
      if img_path.split(".")[-1] in {'HEIC', 'heic'}:
          heif_file = pyheif.read(img_path)
          reImg = Image.frombytes(
            heif_file.mode,
            heif_file.size,
            heif_file.data,
            "raw",
            heif_file.mode,
            heif_file.stride,
          )
          image = np.array(reImg)
      else: image = face_recognition.load_image_file(img_path)

      face_bounding_boxes = face_recognition.face_locations(image)

      if len(face_bounding_boxes) != 1:
      # If there are no people (or too many people) in a training image, skip the image.
        if verbose:
            print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
      else:
      # Add face encoding for current image to the training set
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img, known_face_locations=face_bounding_boxes)[0]
        encodeListKnown.append(encode)
        classNames.append(class_dir)

  print("Number of training images:", len(classNames))

  unique_elements, counts = np.unique(classNames, return_counts=True)
  print("Student name:", unique_elements)
  print("Number of student:", len(unique_elements))

  # Determine how many neighbors to use for weighting in the KNN classifier
  if n_neighbors is None:
      n_neighbors = int(round(math.sqrt(len(encodeListKnown))))
      if verbose:
          print("Chose n_neighbors automatically:", n_neighbors)

  # Create and train the KNN classifier
  knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
  knn_clf.fit(encodeListKnown, classNames)

  return knn_clf


In [ ]:
# Predict the unknown images:
# Recognizes faces in given image using a trained KNN classifier

def predict(X_img, knn_clf=None, distance_threshold=0.47):

  X_face_locations = face_recognition.face_locations(X_img)

  # If no faces are found in the image, return an empty result.
  if len(X_face_locations) == 0:
    print("Didn't find a face in the image.")
    return []

  # Find encodings for faces in the test iamge
  faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

  # Use the KNN model to find the best matches for the test face
  closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
  print("Closest distances are:", closest_distances)

  are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]
  print("Match with the object? ", are_matches)

  # Predict classes and remove classifications that aren't within the threshold
  return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]



In [ ]:
def show_prediction_labels_on_image(img_path, predictions, output_path):
  #:param img_path: path to image to be recognized
  #:param predictions: results of the predict function

  if img_path.split(".")[-1] in {'HEIC', 'heic'}:
    # Convert the HEIC image to a JPEG image
    heif_file = pyheif.read(img_path)
    reImg = Image.frombytes(
            heif_file.mode,
            heif_file.size,
            heif_file.data,
            "raw",
            heif_file.mode,
            heif_file.stride,
          )
    #X_img = np.array(reImg)
    # Save the converted image as JPEG
    temp = output_path.split(".")
    temp[-1] = '.jpg'
    output_path = "".join(temp)
    reImg.save(output_path, format='JPEG')

    # Open the converted image with PIL
    img_path = output_path

  img = Image.open(img_path)
  draw = ImageDraw.Draw(img)

  for name, (top, right, bottom, left) in predictions:
    # Draw a box around the face using the Pillow module
    draw.rectangle(((left, top), (right, bottom)), outline=(255, 105, 180),  width=2)

    # Encode the name to handle non-UTF-8 text
    name = name.encode("UTF-8").decode("latin1")

    # Draw a label with a name below the face
    # Calculate text width and height
    text_bbox = draw.textbbox((left + 6, bottom - 5), name)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    # Draw a label with a name below the face
    draw.rectangle([(left, bottom - text_height - 10), (left + text_width + 12, bottom)], fill=(255, 105, 180), outline=(255, 105, 180))
    draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

    # Remove the drawing library from memory as per the Pillow docs
  del draw

  # Save the resulting image
  # outputWithImage_path = os.path.join(output_path, "_with_text.jpg")
  img.save(output_path)

**Main Function**

In [ ]:
# TRAIN THE MODEL:
# Train the KNN classifier and get the KNN clf for later use.

# import the training images from the path
trainPath = 'drive/My Drive/SCU COEN 240 Machine Learning/trainingList'

print("Training KNN classifier...")
# call the train function
classifier = train(trainPath, n_neighbors=2)
print("Training complete!")

Training KNN classifier...
Image drive/My Drive/SCU COEN 240 Machine Learning/trainingList/Anirudh/0921-ClassPhotos .png not suitable for training: Found more than one face
Image drive/My Drive/SCU COEN 240 Machine Learning/trainingList/Kaiyue/167f63c69e174e2ff4b596ec38b2b94.png not suitable for training: Didn't find a face
Image drive/My Drive/SCU COEN 240 Machine Learning/trainingList/Yutong/1696544875.703294.jpg not suitable for training: Didn't find a face
Image drive/My Drive/SCU COEN 240 Machine Learning/trainingList/Yutong/1697605755.541178.jpg not suitable for training: Didn't find a face
Image drive/My Drive/SCU COEN 240 Machine Learning/trainingList/Yutong/1697767552.7825.jpg not suitable for training: Didn't find a face
Number of training images: 116
Student name: ['Anirudh' 'Aparnaa' 'Chen' 'Dheeraj' 'Divyanth' 'Griffin' 'Haisong'
 'Haochen' 'Huiyu' 'Husain' 'Kaiyue' 'Kimsong' 'Madhuri' 'Manya' 'Peiqi'
 'Pooja' 'Prachi' 'Qihui' 'Ruthu' 'Samyuktha' 'Shreya Chinthala'
 'Shrey

In [ ]:
# TEST THE MODEL:
# Using the trained classifier, make predictions for unknown images
if __name__ == "__main__":
    print("Let's start the magic!")

    testPath = 'drive/My Drive/SCU COEN 240 Machine Learning/minTest'
    saveImage_path = 'drive/My Drive/SCU COEN 240 Machine Learning/resultImage'
    for image_file in os.listdir(testPath):
        full_file_path = os.path.join(testPath, image_file)

        if not os.path.isfile(full_file_path) or os.path.splitext(full_file_path)[1][1:] not in ALLOWED_EXTENSIONS:
          raise Exception("Invalid image path: {}".format(full_file_path))

        if image_file.split(".")[-1] in {'HEIC', 'heic'}:
          heif_file = pyheif.read(full_file_path)
          reImg = Image.frombytes(
            heif_file.mode,
            heif_file.size,
            heif_file.data,
            "raw",
            heif_file.mode,
            heif_file.stride,
          )
          X_img = np.array(reImg)

        else:
          X_img = face_recognition.load_image_file(full_file_path)

        print("---------------------------------------------")
        print("Looking for faces in [{}]".format(image_file))

        # Find all people in the image using a trained classifier model
        predictions = predict(X_img, knn_clf=classifier)
        print("Predict data: ", predictions)
        if len(predictions) == 0:
          continue

        # Print results on the console
        for name, (top, right, bottom, left) in predictions:
            print("【RESULT】- Found {} at ({}, {})".format(name, left, top))
        print(" ")

        # Display recognised person's name on the image
        imageToDraw_path = os.path.join(testPath, image_file)
        imageToSave_path = os.path.join(saveImage_path, image_file)
        show_prediction_labels_on_image(imageToDraw_path, predictions, imageToSave_path)
        print("Result image saved successfully!")
        print("---------------------------------------------")

Let's start the magic!
---------------------------------------------
Looking for faces in [IMG_8271.jpeg]
Closest distances are: (array([[0.21048025]]), array([[79]]))
Match with the object?  [True]
Predict data:  [('Yash', (32, 151, 140, 44))]
【RESULT】- Found Yash at (44, 32)
 
Result image saved successfully!
---------------------------------------------
---------------------------------------------
Looking for faces in [Nt5.png]
Didn't find a face in the image.
Predict data:  []
---------------------------------------------
Looking for faces in [Nt2.jpg]
Didn't find a face in the image.
Predict data:  []
---------------------------------------------
Looking for faces in [Screenshot 2023-10-21 at 9.59.56 PM.png]
Closest distances are: (array([[0.36782069]]), array([[6]]))
Match with the object?  [True]
Predict data:  [('Anirudh', (55, 101, 107, 49))]
【RESULT】- Found Anirudh at (49, 55)
 
Result image saved successfully!
---------------------------------------------
------------------